# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats as st
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_0.head(10)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


In [ ]:
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_1.head(10)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


In [ ]:
data_2 = pd.read_csv('/datasets/geo_data_2.csv')
data_2.head(10)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


In [ ]:
data_0.info(), data_1.info(), data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

(None, None, None)

In [ ]:
data_0.duplicated().sum(), data_1.duplicated().sum(), data_2.duplicated().sum()

(0, 0, 0)

In [ ]:
data_0.isna().sum(),data_1.isna().sum(),data_2.isna().sum()

(id         0
 f0         0
 f1         0
 f2         0
 product    0
 dtype: int64,
 id         0
 f0         0
 f1         0
 f2         0
 product    0
 dtype: int64,
 id         0
 f0         0
 f1         0
 f2         0
 product    0
 dtype: int64)

**Вывод**

В данных не обнаружено пропусков и дубликатов, поэтому можно приступить к дальнейшим действиям.

## Обучение и проверка модели

Разобьем выборки на валидационную и обучающую в соотношении 25:75

In [ ]:
def function(data):

    target = data['product']
    features = data.drop(['product', 'id'], axis=1)

    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
    numeric = ['0', '1', '2']
    scaler = StandardScaler()
    scaler.fit(features_train)
    features_train[numeric] = scaler.transform(features_train)
    features_valid[numeric] = scaler.transform(features_valid)

    return features_train, features_valid, target_train, target_valid

In [ ]:
features_train0, features_valid0, target_train0, target_valid0 = function(data_0)
features_train1, features_valid1, target_train1, target_valid1 = function(data_1)
features_train2, features_valid2, target_train2, target_valid2 = function(data_2)

Теперь обучим модель

In [ ]:
def train(features_train, features_valid, target_train, target_valid):
    model = LinearRegression()
    model.fit(features_train,target_train)
    predictions_valid = pd.Series(model.predict(features_valid))
    print("RMSE =", mean_squared_error(target_valid,predictions_valid) ** 0.5)
    print('Cредний запас предсказанного сырья',predictions_valid.mean())

    return predictions_valid

In [ ]:
predictions_valid0 =  train(features_train0, features_valid0, target_train0, target_valid0)

RMSE = 37.5794217150813
Cредний запас предсказанного сырья 92.59256778438035


In [ ]:
predictions_valid1 =  train(features_train1, features_valid1, target_train1, target_valid1)

RMSE = 0.8930992867756169
Cредний запас предсказанного сырья 68.728546895446


In [ ]:
predictions_valid2 =  train(features_train2, features_valid2, target_train2, target_valid2)

RMSE = 40.02970873393434
Cредний запас предсказанного сырья 94.96504596800489


**Вывод**

Исходя из данных обучения модели, можно сделать вывод о том, что точнее всего 2 датасет, поскольку в 1 и 3 датасете мы можем наблюдать слишком большое значение rmse

## Подготовка к расчёту прибыли

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

In [ ]:
budget = 10000000000
income = 450000
all_well = 500
best_well = 200
revenue = 450000
bootstrap = 1000

In [ ]:
product_volume = budget / income / best_well

In [ ]:
print('Необходимый объем сырья для безубыточной разработки 1 скважины:', product_volume)

Необходимый объем сырья для безубыточной разработки 1 скважины: 111.11111111111111


**Вывод**

Мы получили, что нам необходим объем сырья для безубыточной разработки 111 тыс. баррелей. Но средние значения запасоывв сырья для всех регионов ниже этого объема, поэтому имеется высокая доля риска не окупить затраченный бюджет.

**Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели**

In [ ]:
def profit_calculation(target_valid, predictions_valid, best_well):
    choice = predictions_valid.sort_values(ascending=False)
    deposit = target_valid[choice.index][:best_well]
    income = deposit.sum() * revenue - budget
    return income

## Расчёт прибыли и рисков

In [ ]:
def profit_and_risks (predictions_valid, target_valid, bootstrap, all_well):
    state = np.random.RandomState(12345)
    values = []

    for i in range(bootstrap):
        target_sample = target_valid.reset_index(drop=True).sample(all_well, replace=True, random_state=state)
        predictions_sample = predictions_valid[target_sample.index]
        values.append(profit_calculation(target_sample, predictions_sample, best_well))

    values = pd.Series(values)
    print("Средняя выручка:", values.mean())
    print("95%-й Доверительный интервал:", values.quantile(0.025), '-', values.quantile(0.975))
    print("Риск убытков:", st.percentileofscore(values, 0))

In [ ]:
profit_and_risks(predictions_valid0, target_valid0, bootstrap, all_well)

Средняя выручка: 425938526.91059244
95%-й Доверительный интервал: -102090094.83793654 - 947976353.3583689
Риск убытков: 6.0


In [ ]:
profit_and_risks(predictions_valid1, target_valid1, bootstrap, all_well)

Средняя выручка: 515222773.4432899
95%-й Доверительный интервал: 68873225.37050176 - 931547591.2570494
Риск убытков: 1.0


In [ ]:
profit_and_risks(predictions_valid2, target_valid2, bootstrap, all_well)

Средняя выручка: 435008362.7827556
95%-й Доверительный интервал: -128880547.32978901 - 969706954.1802661
Риск убытков: 6.4


**Вывод**

Таким образом, второй регион имеет наименьшее значение риска, однако регион не подходит по объему. Но несмотря на это, данный регион рекомендуется рассматривать для разработки